In [53]:
import numpy as np
import yfinance as yf
import pandas as pd
import methods.selenium_patch as selenium_patch
from datetime import timedelta
from methods.scraper import Ticker

In [54]:
ticker = "MCFT"
data = Ticker(ticker).macrotrends_key_financial_ratios(frequency="quarterly")
display(data)

field_name,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2025-12-31,2.06640,,,21.63770,3.78910,3.78910,,4.68090,3.52150,0.27630,1.81400,12.30860,7.31200,1.31460,1.91650,0.95810,1.31460,11.61960,0.9595,0.8591
2025-09-30,1.92330,,,22.31240,5.47520,5.47520,,6.58970,5.26940,0.26610,1.60320,6.29410,14.29920,1.96850,2.90540,1.40990,1.96850,11.40140,-0.42060,-0.61010
2025-06-30,1.85880,,,23.15260,7.45520,7.45510,,8.49010,7.16460,0.30590,2.00550,18.51790,4.86020,2.96970,4.42380,2.09730,2.96970,11.18950,1.2379,1.0807
2025-03-31,1.81260,,,20.75430,5.41600,5.41600,,6.41650,4.92760,0.29000,1.52220,8.14240,11.05330,2.09640,3.14550,1.45900,2.09640,10.90640,0.3816,0.2599
2024-12-31,1.84310,,,17.18850,0.29350,0.29350,,1.10620,4.33660,0.25340,1.41870,8.65090,10.40350,0.23860,0.36330,0.17030,0.23860,10.64180,0.389,0.2446
2024-09-30,2.00600,0.21990,0.28180,18.05110,1.53610,1.53610,,1.84980,-7.87190,0.21140,1.43610,4.41230,20.39770,0.57840,0.89160,0.32860,0.45130,10.44520,0.14530,0.01200
2024-06-30,1.98980,0.19620,0.26790,15.84160,-1.08980,-1.08980,,0.43080,-14.79410,0.17080,1.23640,4.54400,19.80630,0.16590,0.25050,0.09590,0.13330,10.97200,-0.5821,-0.7302
2024-03-31,2.01690,0.18920,0.25550,23.33020,8.04030,8.04030,,8.79760,4.47150,0.24640,1.55400,6.23300,14.43930,2.90570,4.25780,1.68010,2.35600,11.58170,0.2219,0.0327
2023-12-31,2.11300,0.19580,0.26610,23.32930,11.99220,11.99220,,12.61730,6.55820,0.27050,1.59820,10.21510,8.81050,4.48970,6.66220,2.61610,3.61060,11.34970,1.6233,1.5209
2023-09-30,2.08070,0.20070,0.27400,23.83230,11.19030,11.19030,,11.69290,6.56910,0.28220,1.29490,5.78420,15.55970,4.44560,6.64390,2.55240,3.55320,11.12050,-0.52960,-0.70770


In [55]:
prices = yf.Ticker(ticker).history("max").get("Close")
earning_prices = []
for date in data.index:
    got_price = False
    i = 0
    while got_price == False and i<7:
        try:
            earning_prices.append(prices[pd.to_datetime(date).tz_localize("America/New_York") - timedelta(days=i)]) # type: ignore
            got_price = True
        except:
            i += 1
    if got_price == False:
        earning_prices.append(np.nan)

earning_changes = [np.nan]
for i in range(1, len(earning_prices)):
    earning_changes.append((earning_prices[i-1]/earning_prices[i]-1)*100)

In [56]:
data["Future Change%"] = earning_changes
data["Price"] = earning_prices
data["Ticker"] = ticker
data = data[data.columns.to_list()[::-1]]
data

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,...,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBITDA Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-12-31,MCFT,18.910000,NaN,0.8591,0.9595,11.61960,1.31460,0.95810,1.91650,1.31460,...,0.27630,3.52150,4.68090,,3.78910,3.78910,21.63770,,,2.06640
2025-09-30,MCFT,21.459999,-11.882569,-0.61010,-0.42060,11.40140,1.96850,1.40990,2.90540,1.96850,...,0.26610,5.26940,6.58970,,5.47520,5.47520,22.31240,,,1.92330
2025-06-30,MCFT,18.580000,15.500534,1.0807,1.2379,11.18950,2.96970,2.09730,4.42380,2.96970,...,0.30590,7.16460,8.49010,,7.45510,7.45520,23.15260,,,1.85880
2025-03-31,MCFT,17.219999,7.897797,0.2599,0.3816,10.90640,2.09640,1.45900,3.14550,2.09640,...,0.29000,4.92760,6.41650,,5.41600,5.41600,20.75430,,,1.81260
2024-12-31,MCFT,19.070000,-9.701103,0.2446,0.389,10.64180,0.23860,0.17030,0.36330,0.23860,...,0.25340,4.33660,1.10620,,0.29350,0.29350,17.18850,,,1.84310
2024-09-30,MCFT,18.209999,4.722683,0.01200,0.14530,10.44520,0.45130,0.32860,0.89160,0.57840,...,0.21140,-7.87190,1.84980,,1.53610,1.53610,18.05110,0.28180,0.21990,2.00600
2024-06-30,MCFT,18.879999,-3.548729,-0.7302,-0.5821,10.97200,0.13330,0.09590,0.25050,0.16590,...,0.17080,-14.79410,0.43080,,-1.08980,-1.08980,15.84160,0.26790,0.19620,1.98980
2024-03-31,MCFT,23.719999,-20.404723,0.0327,0.2219,11.58170,2.35600,1.68010,4.25780,2.90570,...,0.24640,4.47150,8.79760,,8.04030,8.04030,23.33020,0.25550,0.18920,2.01690
2023-12-31,MCFT,22.639999,4.770318,1.5209,1.6233,11.34970,3.61060,2.61610,6.66220,4.48970,...,0.27050,6.55820,12.61730,,11.99220,11.99220,23.32930,0.26610,0.19580,2.11300
2023-09-30,MCFT,22.219999,1.890189,-0.70770,-0.52960,11.12050,3.55320,2.55240,6.64390,4.44560,...,0.28220,6.56910,11.69290,,11.19030,11.19030,23.83230,0.27400,0.20070,2.08070


In [ ]:
def imputer(df: pd.DataFrame) -> pd.DataFrame:
    symbol = df.loc[df.index[0], "Ticker"]
    df = df.drop("Ticker", axis=1)
    max_nans_share = 0.30

    # go through all data points and count nans
    row_nans = [0 for _ in df.index]
    col_nans = [0 for _ in df.columns]
    i = 0
    for row in df.index:
        j = 0
        for column in df.columns:
            val = df.loc[row, column]
            if val == '':
                val = np.nan
                df.loc[row, column] = np.nan
            if np.isnan(float(val)): # type: ignore
                row_nans[i] += 1
                col_nans[j] += 1
            else:
                df.loc[row, column] = float(df.loc[row, column]) # type: ignore
            j += 1
        i += 1
                
    # delete rows and columns with too many nans or missing price
    drop_rows = []
    for i in range(len(df.index)):
        if row_nans[i]/len(df.columns) > max_nans_share or np.isnan(df.loc[df.index[i], "Price"]): # type: ignore
            drop_rows.append(df.index[i])
    drop_cols = []
    for j in range(len(df.columns)):
        if col_nans[j]/len(df.index) > max_nans_share:
            drop_cols.append(df.columns[j])
    df = df.drop(drop_rows)
    df = df.drop(drop_cols, axis=1)
    
    df.loc
    display(df)

    # impute last nan values
    for j in range(len(df.columns)):
        impute_indices = []
        for i in range(len(df.index)):
            if np.isnan(df.iloc[i, j]): # type: ignore
                impute_indices.append(i)
                u = i-1
                l = i+1
                while u <= 0 and l > len(df.index) and np.isnan(df.iloc[i, j]) == False: # type: ignore
                    if np.isnan(df.iloc[u, j]) == False and np.isnan(df.iloc[l, j]) == False: # type: ignore
                        for k in impute_indices:
                            df.iloc[k, j] = (df.iloc[u, j]+df.iloc[l, j])/2 # type: ignore
                    if np.isnan(df.iloc[u, j]): # type: ignore
                        impute_indices.append[u] # type: ignore
                    else:
                        u += -1
                    if np.isnan(df.iloc[l, j]): # type: ignore
                        impute_indices.append[l] # type: ignore
                    else:
                        l += 1


    df.insert(0, "Ticker", symbol)
    return df

imputer(data)

field_name,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,Days Sales In Receivables,...,Inventory Turnover Ratio,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-12-31,18.910000,NaN,0.8591,0.9595,11.6196,1.3146,0.9581,1.9165,1.3146,7.312,...,1.814,0.2763,3.5215,4.6809,3.7891,3.7891,21.6377,NaN,NaN,2.0664
2025-09-30,21.459999,-11.882569,-0.6101,-0.4206,11.4014,1.9685,1.4099,2.9054,1.9685,14.2992,...,1.6032,0.2661,5.2694,6.5897,5.4752,5.4752,22.3124,NaN,NaN,1.9233
2025-06-30,18.580000,15.500534,1.0807,1.2379,11.1895,2.9697,2.0973,4.4238,2.9697,4.8602,...,2.0055,0.3059,7.1646,8.4901,7.4551,7.4552,23.1526,NaN,NaN,1.8588
2025-03-31,17.219999,7.897797,0.2599,0.3816,10.9064,2.0964,1.459,3.1455,2.0964,11.0533,...,1.5222,0.29,4.9276,6.4165,5.416,5.416,20.7543,NaN,NaN,1.8126
2024-12-31,19.070000,-9.701103,0.2446,0.389,10.6418,0.2386,0.1703,0.3633,0.2386,10.4035,...,1.4187,0.2534,4.3366,1.1062,0.2935,0.2935,17.1885,NaN,NaN,1.8431
2024-09-30,18.209999,4.722683,0.012,0.1453,10.4452,0.4513,0.3286,0.8916,0.5784,20.3977,...,1.4361,0.2114,-7.8719,1.8498,1.5361,1.5361,18.0511,0.2818,0.2199,2.006
2024-06-30,18.879999,-3.548729,-0.7302,-0.5821,10.972,0.1333,0.0959,0.2505,0.1659,19.8063,...,1.2364,0.1708,-14.7941,0.4308,-1.0898,-1.0898,15.8416,0.2679,0.1962,1.9898
2024-03-31,23.719999,-20.404723,0.0327,0.2219,11.5817,2.356,1.6801,4.2578,2.9057,14.4393,...,1.554,0.2464,4.4715,8.7976,8.0403,8.0403,23.3302,0.2555,0.1892,2.0169
2023-12-31,22.639999,4.770318,1.5209,1.6233,11.3497,3.6106,2.6161,6.6622,4.4897,8.8105,...,1.5982,0.2705,6.5582,12.6173,11.9922,11.9922,23.3293,0.2661,0.1958,2.113
2023-09-30,22.219999,1.890189,-0.7077,-0.5296,11.1205,3.5532,2.5524,6.6439,4.4456,15.5597,...,1.2949,0.2822,6.5691,11.6929,11.1903,11.1903,23.8323,0.274,0.2007,2.0807


field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,...,Inventory Turnover Ratio,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-12-31,MCFT,18.910000,NaN,0.8591,0.9595,11.6196,1.3146,0.9581,1.9165,1.3146,...,1.814,0.2763,3.5215,4.6809,3.7891,3.7891,21.6377,NaN,NaN,2.0664
2025-09-30,MCFT,21.459999,-11.882569,-0.6101,-0.4206,11.4014,1.9685,1.4099,2.9054,1.9685,...,1.6032,0.2661,5.2694,6.5897,5.4752,5.4752,22.3124,NaN,NaN,1.9233
2025-06-30,MCFT,18.580000,15.500534,1.0807,1.2379,11.1895,2.9697,2.0973,4.4238,2.9697,...,2.0055,0.3059,7.1646,8.4901,7.4551,7.4552,23.1526,NaN,NaN,1.8588
2025-03-31,MCFT,17.219999,7.897797,0.2599,0.3816,10.9064,2.0964,1.459,3.1455,2.0964,...,1.5222,0.29,4.9276,6.4165,5.416,5.416,20.7543,NaN,NaN,1.8126
2024-12-31,MCFT,19.070000,-9.701103,0.2446,0.389,10.6418,0.2386,0.1703,0.3633,0.2386,...,1.4187,0.2534,4.3366,1.1062,0.2935,0.2935,17.1885,NaN,NaN,1.8431
2024-09-30,MCFT,18.209999,4.722683,0.012,0.1453,10.4452,0.4513,0.3286,0.8916,0.5784,...,1.4361,0.2114,-7.8719,1.8498,1.5361,1.5361,18.0511,0.2818,0.2199,2.006
2024-06-30,MCFT,18.879999,-3.548729,-0.7302,-0.5821,10.972,0.1333,0.0959,0.2505,0.1659,...,1.2364,0.1708,-14.7941,0.4308,-1.0898,-1.0898,15.8416,0.2679,0.1962,1.9898
2024-03-31,MCFT,23.719999,-20.404723,0.0327,0.2219,11.5817,2.356,1.6801,4.2578,2.9057,...,1.554,0.2464,4.4715,8.7976,8.0403,8.0403,23.3302,0.2555,0.1892,2.0169
2023-12-31,MCFT,22.639999,4.770318,1.5209,1.6233,11.3497,3.6106,2.6161,6.6622,4.4897,...,1.5982,0.2705,6.5582,12.6173,11.9922,11.9922,23.3293,0.2661,0.1958,2.113
2023-09-30,MCFT,22.219999,1.890189,-0.7077,-0.5296,11.1205,3.5532,2.5524,6.6439,4.4456,...,1.2949,0.2822,6.5691,11.6929,11.1903,11.1903,23.8323,0.274,0.2007,2.0807
